In [ ]:
import numpy as np
import os
import pandas as pd
import pickle
from zipfile import ZipFile

# mltoolz doesn't work for the rest of us because we don't have the file, 
# you can add it to the 'utils' folder and uncomment the 'Access to "utils"' lines

# Access to "utils"
# src_path = os.path.dirname(os.getcwd())
# if os.path.exists(src_path) and src_path not in sys.path:
#     sys.path.append(src_path)

import mltoolz as mlt # not sure if this will work for others 

from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

ModuleNotFoundError: No module named 'mltoolz'

In [16]:
ZIP_PATH = '../data/dataset.zip'

with ZipFile(ZIP_PATH, 'r') as z:
    with z.open(z.namelist()[0]) as f:
        df = pd.read_csv(f, parse_dates=['date'])

In [17]:
df

,date,cloud_cover,sunshine,global_radiation,max_temp,mean_temp,min_temp,precipitation,pressure,snow_depth
0,1979-01-01,2.0,7.0,52.0,2.3,-4.1,-7.5,0.4,101900.0,9.0
1,1979-01-02,6.0,1.7,27.0,1.6,-2.6,-7.5,0.0,102530.0,8.0
2,1979-01-03,5.0,0.0,13.0,1.3,-2.8,-7.2,0.0,102050.0,4.0
3,1979-01-04,8.0,0.0,13.0,-0.3,-2.6,-6.5,0.0,100840.0,2.0
4,1979-01-05,6.0,2.0,29.0,5.6,-0.8,-1.4,0.0,102250.0,1.0
...,...,...,...,...,...,...,...,...,...,...
15336,2020-12-27,1.0,0.9,32.0,7.5,7.5,7.6,2.0,98000.0,NaN
15337,2020-12-28,7.0,3.7,38.0,3.6,1.1,-1.3,0.2,97370.0,NaN
15338,2020-12-29,7.0,0.0,21.0,4.1,2.6,1.1,0.0,98830.0,NaN
15339,2020-12-30,6.0,0.4,22.0,5.6,2.7,-0.1,0.0,100200.0,NaN
